In [ ]:
import numpy as np
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import names
import sys
sns.set(style="white")


In [ ]:
dis_threshold = 5
trans_possibility = 0.5
Total_time = 1 * 60 * 1000
Trans_time = 5 * 1000
Rest_time = 5 * 1000
Listening_time = 0.5 * 1000
Sending_time = 0.5 * 1000
base_name = 1
time = 0

In [ ]:
class State:
    def __init__(self):
        self.IDLE = "IDLE"
        self.LISTENING = "LISTENING"
        self.SENDING = "SENDING"

state = State()

In [ ]:
class Teacher:
    def __init__(self, x = None, y = None):
        self.x = x
        self.y = y
        self.identity = 'Teacher'
        self.label = 0
        self.teacher_listening_time = 0.5 * 1000
        self.teacher_sending_time = 0.5 * 1000
        self.cur_state = state.SENDING
        self.relay = []
        
    def update_label(self):
        # it has label
        module = time % (self.teacher_listening_time + self.teacher_sending_time)
        if module <= self.teacher_listening_time:
            self.cur_state = state.LISTENING
        else:
            self.cur_state = state.SENDING
            
    def listen(self, students):
        if self.cur_state == state.LISTENING:
            for s in students:
                if ((s.label == 1) and (s.cur_state == state.SENDING)):
                    if (np.random.rand() < trans_possibility):
                        self.relay.append(s.output_name())
        else:
            return

In [ ]:
class Seat:
    def __init__(self, x = None, y = None, proba = 0, anc = None, ident = 'Empty', l = None):
        self.x = x
        self.y = y
        global base_name
        self.probability = proba
        if anc is None:
            self.ancestor = []
        else:
            self.ancestor = anc
        self.identity = ident # identity: Empty, Teacher, Student
        self.label = l
        self.cur_state = state.LISTENING # IDLE, LISTENING, SENDING
        self.start_time_bias = np.random.randint(1000)
        self.start_time = None
        self.relay = [names.get_full_name()]
        self.output_index = 0
        
    def output_name(self):
        self.output_index += 1
        return self.relay[self.output_index % len(self.relay)]
    
    def set_x(self, x):
        self.x = x

    def set_y(self, y):
        self.y = y
    
    def set_pros(self, p):
        self.probability = p

    def set_ancestor(self, anc):
        if type(anc) is not list:
            self.ancestor = [anc]
        else:
            self.ancestor = anc
    
    def add_ancestor(self, anc):
        self.ancestor.append(anc)
    
    def set_identity(self, ident):
        self.identity = ident
        
    def set_label(self, l):
        if self.label is None:
            self.start_time = time
        if (l != self.label):
            self.label = l
            self.set_pros(1 - math.pow((1 - math.pow(trans_possibility, l)), (Total_time / (Trans_time + Rest_time))))
            print(str([self.x, self.y]) + " label has been set as " + str(l))
            
        
    def to_string(self):
        print("Seat x:" + str(self.x) + " y:" + str(self.y) + " has ancestor " + str(self.ancestor) + " and label " + str(self.label) + " which is occupied by " + str(self.identity))
        
    def listen(self, students, t_x, t_y):
        if self.cur_state == state.LISTENING:
            if (cal_dis(self.x, self.y, t_x, t_y) < dis_threshold):
                if (np.random.rand() < trans_possibility):
                    self.set_label(1)
            else: 
                for s in students:
                    if s is self:
                        continue
                    if ((s.cur_state == state.SENDING) and (cal_dis(self.x, self.y, s.x, s.y) < dis_threshold)):
                        if (np.random.rand() < trans_possibility):
                            if (self.label is None) or ((s.label < self.label - 1)):
                                self.set_label(s.label + 1)
                                print(str([self.x, self.y]) + " received the signal from " + str([s.x, s.y]))
                            elif (s.label == self.label + 1):
                                print("here!!!!!!!")
                                self.append_relay(s.output_name())
        else:
            return
                          
    def append_relay(self, name):
        l = self.relay
        l.append(name)
        self.relay = list(set(l))
        
    def update_label(self):
        if self.label is None:
            self.cur_state = state.LISTENING
        else:
            # it has label
            if time < (self.start_time + self.start_time_bias):
                self.cur_state = state.LISTENING
            else:
                module = (time - self.start_time_bias - self.start_time) % (Listening_time + Sending_time)
                if module <= Listening_time:
                    self.cur_state = state.LISTENING
                else:
                    self.cur_state = state.SENDING
                        
                        
            
                    
                

In [ ]:
def cal_dis(x1,y1,x2,y2):
    return math.sqrt((x1-x2)*(x1-x2) + (y1-y2)*(y1-y2))

In [ ]:
class Class:

    def __init__(self, length = -1, width= -1):
        self.seat_length = length
        self.seat_width = width
        
        seat_list = []
        for l in range(length):
            for w in range(width):
                seat_list.append(Seat(w,l, ident="Empty"))
        self.class_seats = seat_list
        self.teacher_x = self.seat_width / 2
        self.teacher_y = -1
        self.teacher = Teacher(self.teacher_x,self.teacher_y)
#         self.class_seats.append(Seat(self.teacher_x, self.teacher_y, ident="Teacher"))
        self.max_label = math.ceil(math.sqrt(self.seat_length*self.seat_length + self.seat_width*self.seat_width)/dis_threshold) + 1
        # max_label means the maximum number of label which this class can have
        
    def get_seat(self, x, y):
        return self.class_seats[x + (y * self.seat_width)] # return the Seat OBJECT
    
    def convert_index_to_xy(self,index):
        x = index % self.seat_width
        y = math.floor(index / self.seat_width)
        return x, y
    
    def convert_xy_to_index(self, x, y):
        return x + (y * self.seat_width)
    
    def random_arrange_student(self, student_num):
        """
        Randomly arrange students for this class
        student_number: Number of students in this class
        """
        if len([self.return_all_student_seat()]) > 0:
            # If there are already students in this class
            # Reset all the seats in this class
            self.class_seats = []
            for l in range(self.seat_length):
                for w in range(self.seat_width):
                    self.class_seats.append(Seat(w,l, ident="Empty"))
        arr = np.arange(self.seat_length * self.seat_width)
        np.random.shuffle(arr)
        
        for index in arr[:student_num]:
            self.class_seats[index].set_identity("Student")

    def return_all_student_seat(self):
        res = [] 
        for i in self.class_seats:
            if (i.identity == "Student"):
                res.append(i)
        # res = [Seat1_object, Seat2_object, Seat3_object,....]
        return res
    
    def visualize_students(self):
        """
        Visualize all the student seats in this class
        """
        students = self.return_all_student_seat()
        arr = np.array([[0] * self.seat_width] * self.seat_length) # Initializa a 2d-array with class_width * class_length, initial value is 0
        for s in students:
            arr[s.y][s.x] = 1
            
        # Draw the heatmap
        plt.figure(figsize=(self.seat_length/2, self.seat_width/2))
        sns.heatmap(pd.DataFrame(arr).sort_index(ascending=False), square=True, linewidths=0.5, cmap="YlGnBu")
        plt.show()
        return arr
    
    def visualize_selected_seats(self, seats_list):
        arr = np.array([[0] * self.seat_width] * self.seat_length)
        for s in seats_list:
            arr[s.y][s.x] = 1
        
        plt.figure(figsize=(self.seat_length/2, self.seat_width/2))
        sns.heatmap(pd.DataFrame(arr).sort_index(ascending=False), square=True, linewidths=0.5, cmap="YlGnBu")
        plt.show()
        return arr
    
    def set_ancestor(self):
        """
        Set ancestor for all the student seats in this class
        """
        students = self.return_all_student_seat()
        students_label = np.array([i.label for i in students])
        if (all(x == None for x in students_label)): # if the student labels are all None
            print("Set label 1 and ancestor to teacher for students who is near to teacher")
            for i in students:
                if cal_dis(i.x,i.y,self.teacher_x,self.teacher_y) < dis_threshold:
#                     print("(Teacher nearby) Set label for strudent" + str([i.x, i.y]))
                    i.add_ancestor("Teacher")
#                     print(i.to_string())
                    i.set_label(1)

        count = 0
        while (True):
            students_without_ancestor = [i for i in students if len(i.ancestor) == 0]
            students_with_ancestor = [i for i in students if len(i.ancestor) != 0]
            if len(students_without_ancestor) == 0:
                print("All the students have ancestors now")
                break
            if count > self.max_label:
                print(students_without_ancestor)
                print("There are students cannot be reached by anyone!!!!!")
                break

            for a in students_without_ancestor:
                nearby = []
                for b in students_with_ancestor:
                    if cal_dis(a.x,a.y,b.x,b.y) < dis_threshold:
                        nearby.append(b)
                if len(nearby) > 0:
                    ancestor_label = min([j.label for j in nearby])
                    print("Set label for strudent" + str([a.x, a.y]))
                    a.set_label(ancestor_label + 1)
                    for i in nearby:
                        if (i.label == ancestor_label):
                            a.add_ancestor('Stu' + str([i.x, i.y]))

            count += 1
            
    def visualize_seats_label(self):
        students = self.return_all_student_seat()
        arr = np.array([[0] * self.seat_width] * self.seat_length)
        for s in students:
            if s.label is None:
#                 print("Student " + str(s.x) + ", " + str(s.y) + " cannot be reached by Teacher")
                arr[s.y][s.x] = self.max_label + 1
            else:
                arr[s.y][s.x] = s.label

        plt.figure(figsize=(self.seat_length/2, self.seat_width/2))
        sns.heatmap(pd.DataFrame(arr).sort_index(ascending=False), 
                    square=True, 
                    linewidths=0.5, 
#                     cmap=sns.cubehelix_palette(self.max_label+1, start=2, rot=1, dark=0, light=.95)
                   cmap = "YlGnBu")
                    
        plt.show()
        
        return arr
    
    def loop_once(self):
        global time
        time += 100
        students = self.return_all_student_seat()
        for s in students:
            s.update_label()
            s.listen(students,self.teacher_x, self.teacher_y)
        self.teacher.update_label()
        self.teacher.listen(students)
        

In [ ]:
A = Class(5, 5)
A.random_arrange_student(20)
# A.visualize_students()

In [ ]:
for i in range(1000):
    A.loop_once()
# A.visualize_seats_label()

In [ ]:
A.visualize_seats_label()

In [ ]:
len(set(A.teacher.relay))

In [ ]:
A_students = A.return_all_student_seat()


In [ ]:
for i in A_students:
    print(i.relay)

In [ ]:
str(2)

In [ ]:
a = A.return_all_student_seat()

In [ ]:
i = a[0]

In [ ]:
a.remove(i)

In [ ]:
len(a)

In [ ]:
a

In [ ]:
for s in A_students:
    print(s.output_index)